In [ ]:
#from command line run: conda activate taccari_ext_env
#pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-sparse  -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-geometric

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import sklearn
from pathlib import Path

In [25]:
#run data_processing - DMG added False as a default for synthetic data since I don't have that.
from data_preprocessing import process_data as process_data

In [26]:
process_data.main()

Processed data file not found. Processing data.
Processing new data.
Current working directory: C:\Users\danielg\PycharmProjects\Taccari_et_al


AssertionError: Preprocessed data directory does not exist.

In [27]:
import sys, pathlib
print("Running process_data from:", pathlib.Path(__file__).resolve())
print("Python executable:", sys.executable)

NameError: name '__file__' is not defined

In [10]:
from pathlib import Path
print(Path.cwd())

C:\Users\danielg\PycharmProjects\Taccari_et_al
